Тестовый файл для удобного запуска и отладки составных частей бота 

In [1]:
%reload_ext autoreload
%autoreload 2

In [26]:
import os
from dotenv import load_dotenv
from tinkoff.invest import Client

from tinkoff.invest import CandleInterval
from datetime import datetime

load_dotenv()

TOKEN = os.getenv("INVEST_TOKEN")

TICKER = 'RNFT'

In [3]:
from trader_bot import ScalpingBot

In [39]:
year, month, day, tmz = 2024, 3, 15, 3

date_from = datetime(year, month, day, 9 - tmz, 55)
date_to = datetime(year, month, day, 19 - tmz, 0)

In [41]:
from test_env.time_test_env import TimeTestEnv

bot = ScalpingBot(
    TOKEN, 
    TICKER,
    time_helper=TimeTestEnv(date_from)
    )

2024-03-16 02:23:44,233 - INFO - INIT 
     figi - BBG00F9XX7H4 (RNFT)
     candles_count - 4
     min profit - 5 steps * 0.1 = 0.5
     stop_loss_percent - 1.0 %
     commission - 0.05 %
     no_operation_timeout_seconds - 300 sec
     sleep_trading - 420 sec
     sleep_no_trade - 420 sec



In [5]:
# bot.db_add_deal(1, 10, 1, 9)

In [49]:
with Client(TOKEN) as client:
    candles_1_min = client.market_data.get_candles(
        figi=bot.client.figi,
        from_=date_from,
        to=date_to,
        interval=CandleInterval.CANDLE_INTERVAL_1_MIN
    )
    
    candles_5_min = client.market_data.get_candles(
        figi=bot.client.figi,
        from_=date_from,
        to=date_to,
        interval=CandleInterval.CANDLE_INTERVAL_5_MIN
    )

In [50]:
from tinkoff.invest import Quotation


def f2q(price) -> Quotation:
    return Quotation(units=int(price), nano=int((round(price - int(price), 1)) * 1e9))

def q2f(quotation: Quotation, digits=1):
    return round(quotation.units + quotation.nano * 1e-9, digits)

In [51]:
candles_1_min_dict = {(candle.time.hour + tmz, candle.time.minute): candle for candle in candles_1_min.candles}
candles_5_min_dict = {(candle.time.hour + tmz, candle.time.minute): candle for candle in candles_5_min.candles}

In [54]:
from tinkoff.invest import HistoricCandle
from datetime import datetime, timedelta

def get_n_minutes(h, m, n):
    specified_time = datetime.now().replace(hour=h, minute=m, second=0, microsecond=0)
    prev_minutes = []
    
    # Вычитаем минуты и добавляем результат в список
    for i in range(n):
        previous_time = specified_time + timedelta(minutes=i)
        prev_minutes.append((previous_time.hour, previous_time.minute))
        
    return prev_minutes

# Пример использования
hour, minute, n = 10, 30, 5
t5 = candles_5_min_dict[(hour, minute)]

def get_candle(hour, minute, n = 5):

    previous_minutes = get_n_minutes(hour, minute, n)
    
    open = None
    high = 0
    low = 1000000000
    close = 0
    volume = 0

    for time_pair in previous_minutes:
        t1 = candles_1_min_dict[time_pair]
        if open is None:
            open = t1.open
        high = max(high, q2f(t1.high))
        low  = min(low,  q2f(t1.low))
        close = t1.close
        volume += t1.volume
        
    return HistoricCandle(
        high=f2q(high),
        low=f2q(low),
        open=open,
        close=close,
        volume=volume,
        time=datetime(year, month, day, hour, minute)
    )

def print_candle(candle:HistoricCandle):
    print(f"o{q2f(candle.open)} c{q2f(candle.close)} h{q2f(candle.high)} l{q2f(candle.low)} v{candle.volume}")
    
t5   = candles_5_min_dict[(hour, minute)]     
t5_c = get_candle(hour, minute, n)

print_candle(t5)
print_candle(t5_c)


o174.3 c174.4 h174.5 l173.7 v52270
o174.3 c174.4 h174.5 l173.7 v52270


# Тестовая система

In [55]:
print("Начальная дата и время:", date_from)
print("Конечная дата и время:", date_to)

Начальная дата и время: 2024-03-15 06:55:00
Конечная дата и время: 2024-03-15 16:00:00


In [56]:
# итератор по минутам для заданного времени
def datetime_iterator(start_datetime, end_datetime):
    current_datetime = start_datetime
    while current_datetime <= end_datetime:
        yield current_datetime
        current_datetime += timedelta(minutes=1)

bot.time.set_time(date_from)
bot.time.sleep(0)

# Использование итератора для вывода каждой пары час-минута
for dt in datetime_iterator(date_from, date_to):
    bot.time.set_time(dt)
    if bot.time.is_time_to_awake():
        print(dt.strftime("%H:%M"))
        bot.run_iteration()

06:55
.07:00
.07:05
.07:10
.07:15
.07:20
.07:25
.07:30
07:37
07:44
07:51
07:58
08:05
08:12
08:19
08:26
08:33
08:40
08:47
08:54
09:01
09:08
09:15
09:22
09:29
09:36
09:43
09:50
09:57
10:04
10:11
10:18
10:25
10:32
10:39
10:46
10:53
11:00
11:07
11:14
11:21
11:28
11:35
11:42
11:49
11:56
12:03
12:10
12:17
12:24
12:31
12:38
12:45
12:52
12:59
13:06
13:13
13:20
13:27
13:34
13:41
13:48
13:55
14:02
14:09
14:16
14:23
14:30
14:37
14:44
14:51
14:58
15:05
15:12
15:19
15:26
15:33
15:40
15:47
.15:52
.15:57
.

In [ ]:
# надо сделать итерацию по всем временным промежуткам для бота
# когда включается sleep - мы получаем время до которого алгоритм спит и он не работает до него
